In [1]:
%cd /Users/natalipeeva/Documents/GitHub/Automatic-Answering-of-City-Council-Questions/

/Users/natalipeeva/Documents/GitHub/Automatic-Answering-of-City-Council-Questions


In [2]:
import sys
import os
src_dir = os.path.join(os.getcwd(), 'src')
sys.path.append(src_dir)
from read_data.read_data import read_urls_questions

In [3]:
import pandas as pd
collected, questions =  read_urls_questions(os.path.join('data/all_references/reference_urls_collected.csv'),
                                            os.path.join('data/all_references/questions_updated_urls.csv'),
                                            clean_url_nan=True) 

In [18]:
len(questions)

453

In [4]:
import requests
import io
import PyPDF2
import time
def get_pdf_text(pdf_url):
    """
    Input: PDF URL
    Output: PDF textual content
    """
    pdf = requests.get(pdf_url)
    pdf_content = pdf.content 
    pdf_text = io.BytesIO(pdf_content)
    pdf_reader = PyPDF2.PdfReader(pdf_text)
    text = ""
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text += page.extract_text()
    
    return text

In [5]:
collected.tail()

,Unnamed: 0,URL,Content,Textual_Content,Exception
775,775,https://www.kvk.nl/veilig-zakendoen/wat-kun-je...,"<!DOCTYPE html>\n<html lang=""nl""><head><meta c...",Wat kun je doen tegen ransomware? 6 tips | KVK...,NaN
777,777,https://amsterdam.raadsinformatie.nl/document/...,%PDF-1.7\n\n4 0 obj\n(Identity)\nendobj\n5 0 o...,%PDF-1.7\n\n4 0 obj\n(Identity)\nendobj\n5 0 o...,NaN
778,778,https://amsterdam.raadsinformatie.nl/document/...,%PDF-1.7\r%âãÏÓ\r\n310 0 obj\r&lt;&gt;\rendobj...,%PDF-1.7\r%âãÏÓ\r\n310 0 obj\r<>\rendobj\r ...,NaN
779,779,https://www.at5.nl/artikelen/196980/ruim-730-k...,"<!DOCTYPE html>\n\n<html lang=""nl"">\n<head>\n<...",\n\n\n\n\n\n\nRuim 730 kinderen in daklozenopv...,NaN
780,780,https://amsterdam.raadsinformatie.nl/document/...,%PDF-1.5\n%�쏢\n19 0 obj\n&lt;&gt;\nendobj\n ...,%PDF-1.5\n%�쏢\n19 0 obj\n<>\nendobj\n ...,NaN


In [6]:
for index, row in collected.iterrows():
    if str(row['Textual_Content']).startswith('%PDF'):
        try:
            text = get_pdf_text(row['URL'])
            collected.at[index, 'Textual_Content'] = text
            time.sleep(2)
        except:
            collected.at[index, 'Textual_Content'] = None
            continue

Invalid stream (index 65) within object 146 0: Stream has ended unexpectedly


In [10]:
a = collected.dropna(subset=['Textual_Content'], inplace=True)

In [15]:
collected.reset_index(drop=True, inplace=True)

In [17]:
collected.to_csv('data/all_references/reference_urls_collected_clean_pdf.csv')